In [19]:
import requests
import re
import time
import pandas as pd
import numpy as np
import math

from bs4 import BeautifulSoup
import datetime
from tqdm.notebook import tqdm

In [21]:
RANDOM_STATE=0
tqdm.pandas()
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [ ]:
#https://dateandtime.info/ru/ajax/citysearch.php?query=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0

In [2]:
def season_parser(season, city_code):
    
    def count_light_days(timeList: list):
        mysum = datetime.timedelta()
        for i in timeList:
            (h, m, s) = i.split(':')
            d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
            mysum += d
        return mysum.days
    
    return_dct = {}
    
    month_daytime = []
    month_daytime_for_count = []
    
    for month in season:
        url = f"https://dateandtime.info/ru/citysunrisesunset.php?id={city_code}&month={month}&year=2023"
        try:
            search_engine = requests.get(url)
        except:
            time.sleep(2.5)
            search_engine = requests.get(url)
        soup = BeautifulSoup(search_engine.text)
        search_page =  soup.find_all('table', class_="sunrise_table")[1].find_all('tr')[2:]
        
        for day in search_page:
            
            month_daytime.append(float(re.sub("\n","",re.sub(r"<.*?>",'',str(day.find_all("td")[-1])))[:-4].replace(":", ".")))
            month_daytime_for_count.append(re.sub("\n","",re.sub(r"<.*?>",'',str(day.find_all("td")[-1]))))
            
        return_dct['mean_time'] = float(math.ceil(np.mean(month_daytime)))
        return_dct['count_days'] = count_light_days(month_daytime_for_count)
        
    return return_dct

def time_features_parser(city):
    city=city.replace(" ", "+")
   
    url = f"https://dateandtime.info/ru/ajax/citysearch.php?query={city}"
    try:
        search_engine = requests.get(url)
    except:
        time.sleep(2.5)
        search_engine = requests.get(url)
    soup = BeautifulSoup(search_engine.text)
   
    city_code =  re.findall("RU\.(\d+)",str(soup))[0]
    
    winter = [12,1,2]
    spring = [3,4,5]
    summer = [6,7,8]
    autumn = [9,10,11]
    
    all_seasons = {
        "winter": winter,
        "spring": spring,
        "summer": summer,
        "autumn": autumn,
    }
    
    dct_all_seasons = {}
    
    for season in all_seasons.keys():
        dct_all_seasons[season] = season_parser(all_seasons[season], city_code)
    
    
    
    

    # Тут будет season_parser
    #https://dateandtime.info/ru/citysunrisesunset.php?id=472045&month=10&year=2023
    #sunrise_table
    #search_page =  soup.find_all('a', {'class': 'command'})
    return dct_all_seasons#search_page

In [6]:
time_features_parser("Воронеж")

{'winter': {'mean_time': 9.0, 'count_days': 32},
 'spring': {'mean_time': 14.0, 'count_days': 53},
 'summer': {'mean_time': 16.0, 'count_days': 60},
 'autumn': {'mean_time': 11.0, 'count_days': 40}}

In [14]:
url1 = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
df_city = pd.read_html(url1)[0]
df_city.drop(['№', 'Герб', 'Население', 'Основание илипервое упоминание', 'Статус города[4]', 'Прежние названия'], axis=1, inplace=True)

In [18]:
df_city

,Город,Регион,Федеральный округ,Средний световой день (зима),Количество световых дней (зима),Средний световой день (весна),Количество световых дней (весна),Средний световой день (лето),Количество световых дней (лето),Средний световой день (осень),Количество световых дней (осень)
0,Абаза,Хакасия,Сибирский,,,,,,,,
1,Абакан,Хакасия,Сибирский,,,,,,,,
2,Абдулино,Оренбургская область,Приволжский,,,,,,,,
3,Абинск,Краснодарский край,Южный,,,,,,,,
4,Агидель,Башкортостан,Приволжский,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
1113,Ярославль,Ярославская область,Центральный,,,,,,,,
1114,Ярцево,Смоленская область,Центральный,,,,,,,,
1115,Ясногорск,Тульская область,Центральный,,,,,,,,
1116,Ясный,Оренбургская область,Приволжский,,,,,,,,


In [17]:
df_city['Средний световой день (зима)'] = ''
df_city['Количество световых дней (зима)'] = ''
df_city['Средний световой день (весна)'] = ''
df_city['Количество световых дней (весна)'] = ''
df_city['Средний световой день (лето)'] = ''
df_city['Количество световых дней (лето)'] = ''
df_city['Средний световой день (осень)'] = ''
df_city['Количество световых дней (осень)'] = ''

In [26]:
for index, row in tqdm(df_city.iterrows()):
    city_name = row['Город']
    print(city_name)
    try:
        info = time_features_parser(city_name)
        df_city.at[index, 'Средний световой день (зима)'] = info['winter']['mean_time']
        df_city.at[index, 'Количество световых дней (зима)'] = info['winter']['count_days']
        df_city.at[index, 'Средний световой день (весна)'] = info['spring']['mean_time']
        df_city.at[index, 'Количество световых дней (весна)'] = info['spring']['count_days']
        df_city.at[index, 'Средний световой день (лето)'] = info['summer']['mean_time']
        df_city.at[index, 'Количество световых дней (лето)'] = info['summer']['count_days']
        df_city.at[index, 'Средний световой день (осень)'] = info['autumn']['mean_time']
        df_city.at[index, 'Количество световых дней (осень)'] = info['autumn']['count_days']
    except:
        df_city.at[index, 'Средний световой день (зима)'] = 'Нет информации'
        df_city.at[index, 'Количество световых дней (зима)'] = 'Нет информации'
        df_city.at[index, 'Средний световой день (весна)'] = 'Нет информации'
        df_city.at[index, 'Количество световых дней (весна)'] = 'Нет информации'
        df_city.at[index, 'Средний световой день (лето)'] = 'Нет информации'
        df_city.at[index, 'Количество световых дней (лето)'] = 'Нет информации'
        df_city.at[index, 'Средний световой день (осень)'] = 'Нет информации'
        df_city.at[index, 'Количество световых дней (осень)'] = 'Нет информации'

0it [00:00, ?it/s]

Абаза
Абакан
Абдулино
Абинск
Агидель
Агрыз
Адыгейск
Азнакаево
Азов
Ак-Довурак
Аксай
Алагир
Алапаевск
Алатырь
Алдан
Алейск
Александров
Александровск
Александровск-Сахалинский
Алексеевка
Алексин
Алзамай
Алупкане призн.
Алуштане призн.
Альметьевск
Амурск
Анадырь
Анапа
Ангарск
Андреаполь
Анжеро-Судженск
Анива
Апатиты
Апрелевка
Апшеронск
Арамиль
Аргун
Ардатов
Ардон
Арзамас
Аркадак
Армавир
Армянскне призн.
Арсеньев
Арск
Артём
Артёмовск
Артёмовский
Архангельск
Асбест
Асино
Астрахань
Аткарск
Ахтубинск
Ачинск
Ачхой-Мартан
Аша
Бабаево
Бабушкин
Бавлы
Багратионовск
Байкальск
Баймак
Бакал
Баксан
Балабаново
Балаково
Балахна
Балашиха
Балашов
Балей
Балтийск
Барабинск
Барнаул
Барыш
Батайск
Бахчисарайне призн.
Бежецк
Белая Калитва
Белая Холуница
Белгород
Белебей
Белёв
Белинский
Белово
Белогорск
Белогорскне призн.
Белозерск
Белокуриха
Беломорск
Белоозёрский
Белорецк
Белореченск
Белоусово
Белоярский
Белый
Бердск
Березники
Берёзовский
Берёзовский
Беслан
Бийск
Бикин
Билибино
Биробиджан
Бирск
Бирюсинск
Бирюч

In [30]:
region_mean = {}
regions = sorted(df_city['Регион'].unique())
regions

['Адыгея',
 'Алтай',
 'Алтайский край',
 'Амурская область',
 'Архангельская область',
 'Астраханская область',
 'Башкортостан',
 'Белгородская область',
 'Брянская область',
 'Бурятия',
 'Владимирская область',
 'Волгоградская область',
 'Вологодская область',
 'Воронежская область',
 'Дагестан',
 'Еврейская АО',
 'Забайкальский край',
 'Ивановская область',
 'Ингушетия',
 'Иркутская область',
 'Кабардино-Балкария',
 'Калининградская область',
 'Калмыкия',
 'Калужская область',
 'Камчатский край',
 'Карачаево-Черкесия',
 'Карелия',
 'Кемеровская область',
 'Кировская область',
 'Коми',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Крым',
 'Курганская область',
 'Курская область',
 'Ленинградская область',
 'Липецкая область',
 'Магаданская область',
 'Марий Эл',
 'Мордовия',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Ненецкий АО',
 'Нижегородская область',
 'Новгородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская обла

In [40]:
for reg in regions:
    dlwin = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Средний световой день (зима)'].to_list() if type(x) != str])
    dcwin = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Количество световых дней (зима)'].to_list() if type(x) != str])
    dlspr = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Средний световой день (весна)'].to_list() if type(x) != str])
    dcspr = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Количество световых дней (весна)'].to_list() if type(x) != str])
    dlsum = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Средний световой день (лето)'].to_list() if type(x) != str])
    dcsum = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Количество световых дней (лето)'].to_list() if type(x) != str])
    dlaut = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Средний световой день (осень)'].to_list() if type(x) != str])
    dcaut = np.mean([x for x in df_city.loc[df_city['Регион'] == reg]['Количество световых дней (осень)'].to_list() if type(x) != str])
    region_mean[reg] = {'dlwin': dlwin, 'dcwin': dcwin, 'dlspr': dlspr, 'dcspr': dcspr, 'dlsum': dlsum, 'dcsum': dcsum, 'dlaut': dlaut, 'dcaut': dcaut}

c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [41]:
region_mean

{'Адыгея': {'dlwin': 10.0,
  'dcwin': 35.0,
  'dlspr': 14.0,
  'dcspr': 51.0,
  'dlsum': 15.0,
  'dcsum': 57.0,
  'dlaut': 11.0,
  'dcaut': 42.0},
 'Алтай': {'dlwin': 9.0,
  'dcwin': 32.0,
  'dlspr': 14.0,
  'dcspr': 53.0,
  'dlsum': 16.0,
  'dcsum': 60.0,
  'dlaut': 11.0,
  'dcaut': 40.0},
 'Алтайский край': {'dlwin': 9.0,
  'dcwin': 32.0,
  'dlspr': 14.0,
  'dcspr': 52.833333333333336,
  'dlsum': 16.0,
  'dcsum': 60.666666666666664,
  'dlaut': 11.0,
  'dcaut': 40.25},
 'Амурская область': {'dlwin': 9.0,
  'dcwin': 32.22222222222222,
  'dlspr': 14.0,
  'dcspr': 52.666666666666664,
  'dlsum': 16.22222222222222,
  'dcsum': 60.22222222222222,
  'dlaut': 11.0,
  'dcaut': 40.55555555555556},
 'Архангельская область': {'dlwin': 6.75,
  'dcwin': 24.583333333333332,
  'dlspr': 15.25,
  'dcspr': 56.833333333333336,
  'dlsum': 18.666666666666668,
  'dcsum': 70.08333333333333,
  'dlaut': 10.0,
  'dcaut': 37.666666666666664},
 'Астраханская область': {'dlwin': 9.666666666666666,
  'dcwin': 34.5,


In [42]:
df1 = df_city.copy()

In [91]:
df_city = df1.copy()

In [92]:
df_city['Количество световых дней в году'] = ''

In [93]:
for index, row in tqdm(df_city.iterrows()):
    reg = row['Регион']
    if row['Средний световой день (зима)'] == 'Нет информации':
        df_city.at[index, 'Средний световой день (зима)'] = region_mean[reg]['dlwin']
        df_city.at[index, 'Количество световых дней (зима)'] = region_mean[reg]['dcwin']
        df_city.at[index, 'Средний световой день (весна)'] = region_mean[reg]['dlspr']
        df_city.at[index, 'Количество световых дней (весна)'] = region_mean[reg]['dcspr']
        df_city.at[index, 'Средний световой день (лето)'] = region_mean[reg]['dlsum']
        df_city.at[index, 'Количество световых дней (лето)'] = region_mean[reg]['dcsum']
        df_city.at[index, 'Средний световой день (осень)'] = region_mean[reg]['dlaut']
        df_city.at[index, 'Количество световых дней (осень)'] = region_mean[reg]['dcaut']
for index, row in tqdm(df_city.iterrows()):
    df_city.at[index, 'Количество световых дней в году'] = row['Количество световых дней (зима)'] + row['Количество световых дней (весна)'] + row['Количество световых дней (лето)'] + row['Количество световых дней (осень)']

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [94]:
df_city.fillna(0, inplace=True)

In [95]:
df_city.to_csv('./docs/day_light_data.csv', index=False)